<a href="https://colab.research.google.com/github/Ludvins/VC/blob/master/Practica%202/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [0]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils
# Import optimizer
from keras.optimizers import SGD
# Import data
from keras.datasets import cifar100
# Import models and layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization, Conv1D
# Import image proprocessors
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# Import Resnet
from keras.applications.resnet50 import ResNet50, preprocess_input
# Import Early Stopping
from keras.callbacks import EarlyStopping
# Show no TensorFlow deprecation warnings
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
# Drive mount
path = "/content/drive/My Drive/VC/"
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
def prepare_files():
  !rm -rf "/content/drive/My Drive/VC/images"
  !rm -rf "/content/drive/My Drive/VC/lists"
  !unzip -q "/content/drive/My Drive/VC/Caltech" -d "/content/drive/My Drive/VC"

In [0]:
prepare_files()

## Auxiliar functions

Loads Cifar data.

In [0]:
def load_cifar_data():
    # Load Data
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    # Transform to float and normalize
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    # We will use only 25 classes.
    train_idx = np.isin(y_train, np.arange(25))
    train_idx = np.reshape(train_idx, -1)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    test_idx = np.isin(y_test, np.arange(25))
    test_idx = np.reshape(test_idx, -1)
    x_test = x_test[test_idx]
    y_test = y_test[test_idx]
    
    # Set labels to cathegorical
    y_train = np_utils.to_categorical(y_train, 25)
    y_test = np_utils.to_categorical(y_test, 25)
    
    return x_train, y_train, x_test, y_test

Returns images and classes in `relative_path`.







In [0]:
def read_img (relative_path):
    classes = np.array([im.split('/')[0] for im in relative_path])
    vim = np.array([img_to_array(load_img(path + "images/"+ im, target_size = (224,224,3))) 
                    for im in relative_path])

    return vim, classes

Reads `Caltech` data in `x_train`, `y_train`, `x_test` y `y_test`

In [0]:
def load_caltech_data():

    # Load files with paths
    train_path = np.loadtxt(path + "lists/train.txt", dtype = str)
    test_path = np.loadtxt(path + "lists/test.txt", dtype = str)

    # Read imgs
    print("Loading data...")
    x_train, y_train = read_img(train_path)
    x_test, y_test = read_img(test_path)
    print("Data loaded.")

    # Set classes to numerical
    unique_classes = np.unique(np.copy(y_train))
    for i in range(len(unique_classes)):
      y_train[y_train == unique_classes[i]] = i
      y_test[y_test == unique_classes[i]] = i

    # Set classes to categorical
    y_train = np_utils.to_categorical(y_train, 200)
    y_test = np_utils.to_categorical(y_test, 200)

    # Shuffle data
    x_train_perm = np.random.permutation(len(x_train))
    x_train = x_train[x_train_perm]
    y_train = y_train[x_train_perm]

    x_test_perm = np.random.permutation(len(x_test))
    x_test = x_test[x_test_perm]
    y_test = y_test[x_test_perm]

    return x_train, y_train, x_test, y_test

In [0]:
def load_preprocessed_caltech_data():
  x_train, y_train, x_test, y_test = load_caltech_data()
  return preprocess_input(x_train), y_train, preprocess_input(x_test), y_test

Draws two graphics.
+ Loss function evolution.
+ Accuracy function evolution.

Args:
+ ```hist```: Training records

In [0]:
def show_evolution(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()
    
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training accuracy', 'Validation accuracy'])
    plt.show()

Draws the same graphics as ```show_evolution``` but using a list of records ```hist```.

In [0]:
def compare_evolution(hist, names):

    for i in hist:
        val_loss = i.history['val_loss']
        plt.plot(val_loss)

    plt.legend(["Validation loss " + names[i] for i in range(len(hist))])
    plt.show()

    for i in hist:
        val_acc = i.history['val_acc']
        plt.plot(val_acc)

    plt.legend(["Validation accuracy " + names[i] for i in range(len(hist))])
    plt.show()

## Data generators

In [0]:
def sample_data_generator():
  return ImageDataGenerator(
      featurewise_center = True,             # set input mean to 0 over the dataset
      samplewise_center = False,             # set each sample mean to 0
      featurewise_std_normalization = True,  # divide inputs by std of the dataset
      samplewise_std_normalization = False,  # divide each input by its std
      zca_whitening = False,                 # apply ZCA whitening
      zca_epsilon = 1e-06,                   # epsilon for ZCA whitening
      rotation_range = 0,                    # randomly rotate images in the range (degrees, 0 to 180)
      width_shift_range = 0.1,               # randomly shift images horizontally (fraction of total width)
      height_shift_range = 0.1,              # randomly shift images vertically (fraction of total height)
      shear_range = 0.,                      # set range for random shear
      zoom_range = 0.,                       # set range for random zoom
      channel_shift_range = 0.,              # set range for random channel shifts
      fill_mode = 'nearest',                 # set mode for filling points outside the input boundaries
      cval = 0.,                             # value used for fill_mode = "constant"
      horizontal_flip = True,                # randomly flip images
      vertical_flip = False,                 # randomly flip images
      rescale = None,                        # set rescaling factor (applied before any other transformation)
      preprocessing_function = None,         # set function that will be applied on each input
      data_format = None,                    # image data format, either "channels_first" or "channels_last"
      validation_split = 0.1)                # fraction of images reserved for validation (strictly between 0 and 1)

In [0]:
def train_data_generator():
    return ImageDataGenerator(
        validation_split = 0.1)

In [0]:
def test_data_generator():
    return ImageDataGenerator()

In [0]:
def train_data_generator_with_whitening():
  return ImageDataGenerator(
      featurewise_center = True,             
      zca_whitening = True,                 
      horizontal_flip = True,             
      validation_split = 0.1)                

In [0]:
def test_data_generator_with_whitening():
  return ImageDataGenerator(
      featurewise_center = True,             
      zca_whitening = True)   

In [0]:
def train_data_generator_without_whitening():
  return ImageDataGenerator(
      featurewise_center = True,             
      featurewise_std_normalization = True, 
      width_shift_range = 0.1,               
      height_shift_range = 0.1,              
      horizontal_flip = True,               
      validation_split = 0.1)              

In [0]:
def test_data_generator_without_whitening():
  return ImageDataGenerator(
      featurewise_center = True,             
      featurewise_std_normalization = True)  

## Model definitions

In [0]:
def simple_base_net_model():
    model = Sequential()
    model.name = "Modelo BaseNet"
    model.add(Conv2D(6, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=(32,32,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=(5,5), 
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='softmax'))
    
    return model

In [0]:
def complex_base_net_model_no_norm():
    model = Sequential()
    model.name = "Modelo BaseNet sin BatchNormalization"
    model.add(Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape=(32,32,3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(25, activation='softmax'))

    return model

In [0]:
def complex_base_net_model_norm_after_relu():
    model = Sequential()
    model.name = "Modelo BaseNet con BatchNormalization despues de Activacion"
    model.add(Conv2D(32, (3, 3), padding='same', activation = 'relu', input_shape=(32,32,3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(25, activation='softmax'))

    return model

In [0]:
def complex_base_net_model_norm_before_relu():
    model = Sequential()
    model.name = "Modelo BaseNet con BatchNormalization antes de Activacion"
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(25, activation='softmax'))

    return model

In [0]:
def basic_resnet(freeze = True):
      base_model = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=(224,224,3))
      if (freeze):
        for layer in base_model.layers:
          layer.trainable = False

      model = Sequential()
      model.name = "ResNet Original"
      model.add(base_model)
      model.add(Dense(200,
                    activation = 'softmax'))
      
      return model

In [0]:
def modified_resnet_model(freeze = True):

    base_model = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=(224,224,3))
    if (freeze):
      for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.name = "ResNet Modificado"
    model.add(base_model)
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(200,
                    activation = 'softmax'))

    return model

In [0]:
def modified_resnet_model2(freeze = True):

    base_model = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=(224,224,3))
    if (freeze):
      for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.name = "ResNet Modificado"
    model.add(base_model)
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.60))
    model.add(Dense(200,
                    activation = 'softmax'))

    return model

In [0]:
def modified_resnet_model_output_input():
      resnet50 = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=(224,224,3))

      x = resnet50.output
      x = Dense(1024, activation='relu')(x)
      last = Dense(200, activation='softmax')(x)
      model = Model(inputs=resnet50.input, output = last)
      
      return model

## Train function

In [0]:
def train_and_evaluate(model, train_data_generator, test_data_generator, load_data_function, batch_size, epochs, verbose):

    if (verbose != 0):
      print(" - ENTRENAMIENTO - ")

    model.compile(
        loss = keras.losses.categorical_crossentropy,
        optimizer = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True),
        metrics = ['accuracy']
    )

    x_train, y_train, x_test, y_test = load_data_function()
    train_iterator = train_data_generator()
    test_iterator = test_data_generator()
    train_iterator.fit(x_train)
    #datagen.standardize(x_test)  https://github.com/keras-team/keras/issues/2559
    test_iterator.fit(x_train)

    h = model.fit_generator(
                generator = train_iterator.flow(x_train, y_train, batch_size, subset='training'),
                steps_per_epoch = len(x_train)*0.9/batch_size,
                epochs = epochs,
                validation_data = train_iterator.flow(x_train, y_train, batch_size, subset='validation'),
                validation_steps = len(x_train)*0.1/batch_size,
                verbose = verbose,
                callbacks = [EarlyStopping(monitor = 'val_acc', patience = 10, restore_best_weights = True)]
                )
    if (verbose != 0):
      print(" - RESULTADOS - ")

    score = model.evaluate_generator(
        generator = test_iterator.flow(x_test, y_test, 1),
        verbose = verbose
        )
    print('PÉRDIDA: ', score[0])
    print('PRECISIÓN: ', score[1])
    return h

# Main functions

In [0]:
print("--- APARTADO 1. ---")
m = simple_base_net_model()
m.summary()
h = train_and_evaluate(m, train_data_generator, test_data_generator, 32, load_cifar_data, 20, 1)
print(" - GRÁFICAS - ")
show_evolution(h)

In [0]:
print("--- COMPARACIÓN DE GENERADORES ---")
print(" - RESULTADOS SIN WHITENING - ")
h2 = train_and_evaluate(simple_base_net_model(), train_data_generator_without_whitening, test_data_generator_without_whitening, 32, load_cifar_data, 20, 0)
print(" - RESULTADOS CON WHITENING - ")
h3 = train_and_evaluate(simple_base_net_model(), train_data_generator_with_whitening, test_data_generator_with_whitening, 32, load_cifar_data, 20, 0)
print(" - GRÁFICAS COMPARATIVAS -")
compare_evolution([h, h2, h3], ["Básico", "Complejo no Whitening", "Complejo Withening"])

In [0]:
#@title
print(" --- BASENET MODIFICADO SIN BATCHNORMALIZATION ---")
m3 = complex_base_net_model_no_norm()
m3.summary()
h3 = train_and_evaluate(m3, complex_data_generator, 32)
print(" - GRÁFICAS DE RESULTADOS - ")
show_evolution(h3)

In [0]:
print(" --- BASENET MODIFICADO CON BATCHNORMALIZATION ANTES VS DESPUES --- ")
print(" -- BATCHNORMALIZATION ANTES -- ")
m3 = complex_base_net_model_norm_before_relu()
h3 = train_and_evaluate(m3, complex_data_generator, 32, verbose = 0)
print(" - GRÁFICAS RESULTADOS - ")
show_evolution(h3)

print(" -- BATCHNORMALIZATION DESPUÉS -- ")
m4 = complex_base_net_model_norm_after_relu()
h4 = train_and_evaluate(m4, complex_data_generator, 32, verbose = 0)
print(" - GRÁFICAS RESULTADOS - ")
show_evolution(h4)

print(" - GRÁFICAS CONJUNTAS - ")
compare_evolution([h3, h4], ["Antes", "Despues"])

In [0]:
print(" -- BATCHNORMALIZATION DESPUÉS Y WHITENING -- ")
m = complex_base_net_model_norm_after_relu()
h = train_and_evaluate(m, data_generator_with_whitening, 32, verbose = 1)
print(" - GRÁFICAS RESULTADOS - ")
show_evolution(h)

In [0]:
print(" --- RESNET SIN MODIFICAR --- ")
m5 = basic_resnet(True)
m5.summary()
h5 = train_and_evaluate(m5, basic_data_generator, 32, load_preprocessed_caltech_data, epochs = 10, verbose = 1)
print(" - GRÁFICAS - ")
show_evolution(h5)

In [0]:
print(" --- RESNET MODIFICADO PRIMERAS CAPAS CONGELADAS ----")
m6 = modified_resnet_model()
m6.summary()
h6 = train_and_evaluate(m6, basic_data_generator, 32, load_preprocessed_caltech_data, epochs = 20, verbose = 1)
print(" - GRÁFICAS - ")
show_evolution(h6)

In [0]:
print(" --- RESNET MODIFICADO SIN CONGELAR --- ")
m = modified_resnet_model(freeze = False)
m.summary()
h = train_and_evaluate(m, basic_data_generator, 32, load_preprocessed_caltech_data, epochs = 10, verbose = 1)
print(" - GRÁFICAS - ")
show_evolution(h)

In [0]:
print(" --- RESNET MODIFICADO SIN CONGELAR UTILIZANDO OUTPUT-INPUT --- ")
m = modified_resnet_model_output_input()
#m.summary()
h = train_and_evaluate(m, basic_data_generator, 32, load_preprocessed_caltech_data, epochs = 10, verbose = 1)
print(" - GRÁFICAS - ")
show_evolution(h)

In [0]:
print(" --- RESNET MODIFICADO SIN CONGELAR MODELO 2 --- ")
m = modified_resnet_model2()
#m.summary()
h = train_and_evaluate(m, basic_data_generator, 32, load_preprocessed_caltech_data, epochs = 10, verbose = 1)
print(" - GRÁFICAS - ")
show_evolution(h)